# Concat, Join and Compare with Pandas

Pandas provides various facilities for easily combining together Series or DataFrame with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

Note: Adding a column to a DataFrame is relatively fast. However, adding a row requires a copy, and may be expensive.

In [1]:
import numpy as np
np.random.seed(0)
import pandas as pd

## Recap: Concatenation of np.ndarrays

In [2]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]

print(np.concatenate([x, y, z]))

[1 2 3 4 5 6 7 8 9]


In [3]:
x = [[1, 2],
     [3, 4]]

print(np.concatenate([x, x], axis=1))

[[1 2 1 2]
 [3 4 3 4]]


## Concatenation with Pandas

Like its sibling function on ndarrays, numpy.concatenate, pandas.concat takes a list or dict of homogeneously-typed objects and concatenates them with some configurable handling of “what to do with the other axes".

```python
pd.concat(
    objs,
    axis=0,
    join="outer",
    copy=True,
    ...
)
```

- objs : a sequence or mapping of Series or DataFrame objects.
- axis : {0, 1, …}, default 0. The axis to concatenate along.
- join : {‘inner’, ‘outer’}, default ‘outer’. How to handle indexes on other axis(es). Outer for union and inner for intersection.

In [4]:
s1 = pd.Series([1, 2, 3], index=['A', 'B', 'C'])
s2 = pd.Series([4, 5, 6], index=['D', 'E', 'F'])

In [5]:
print(pd.concat([s1, s2]))

A    1
B    2
C    3
D    4
E    5
F    6
dtype: int64


In [6]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3]
)

df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7]
)

df3 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "E": ["D4", "D5", "D6", "D7"],
    },
    index=[8, 9, 10, 11]
)

In [7]:
df_concat1 = pd.concat([df1, df2])

print(df_concat1)

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
4  A4  B4  C4  D4
5  A5  B5  C5  D5
6  A6  B6  C6  D6
7  A7  B7  C7  D7


In [8]:
df_concat2 = pd.concat(
    [df1, df3],
    axis=0,
    join="outer"
)

print(df_concat2)

     A   B   C    D    E
0   A0  B0  C0   D0  NaN
1   A1  B1  C1   D1  NaN
2   A2  B2  C2   D2  NaN
3   A3  B3  C3   D3  NaN
8   A4  B4  C4  NaN   D4
9   A5  B5  C5  NaN   D5
10  A6  B6  C6  NaN   D6
11  A7  B7  C7  NaN   D7


In [9]:
df_concat3 = pd.concat(
    [df1, df3],
    axis=0,
    join="inner"
)

print(df_concat3)

     A   B   C
0   A0  B0  C0
1   A1  B1  C1
2   A2  B2  C2
3   A3  B3  C3
8   A4  B4  C4
9   A5  B5  C5
10  A6  B6  C6
11  A7  B7  C7


## Append with Pandas

A useful shortcut to concat() are the append() instance methods on Series and DataFrame. They concatenate along axis=0, namely the index.

In [10]:
df_append = df1.append(df3)

print(df_append)

     A   B   C    D    E
0   A0  B0  C0   D0  NaN
1   A1  B1  C1   D1  NaN
2   A2  B2  C2   D2  NaN
3   A3  B3  C3   D3  NaN
8   A4  B4  C4  NaN   D4
9   A5  B5  C5  NaN   D5
10  A6  B6  C6  NaN   D6
11  A7  B7  C7  NaN   D7


## Joining and Merging Data Frames

Pandas provides a single function, merge(), as the entry point for all standard database join operations between DataFrame or named Series objects.

```python
pd.merge(
    left,
    right,
    how="inner",
    on=None,
    ...
)
````

- left: A DataFrame or named Series object.
- right: Another DataFrame or named Series object.
- on: Column or index level names to join on. Must be found in both the left and right DataFrame and/or Series objects.

#### Brief primer on merge methods (relational algebra)

- one-to-one joins: for example when joining two DataFrame objects on their indexes (which must contain unique values).
- many-to-one joins: for example when joining an index (unique) to one or more columns in a different DataFrame.
- many-to-many joins: joining columns on columns.

| Merge method 	| SQL Join Name 	| Description 	|
|-	|-	|-	|
| left 	| LEFT OUTER JOIN 	| Use keys from left frame only 	|
| right 	| RIGHT OUTER JOIN 	| Use keys from right frame only 	|
| outer 	| FULL OUTER JOIN 	| Use union of keys from both frames 	|
| inner 	| INNER JOIN 	| Use intersection of keys from both frames 	|

In [17]:
left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)


right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

print(left)
print(right)

  key   A   B
0  K0  A0  B0
1  K1  A1  B1
2  K2  A2  B2
3  K3  A3  B3
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3


In [12]:
print(pd.merge(left, right, on="key"))

  key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3


In [15]:
print(left.join(right, how="outer"))

      A    B    C    D
K0   A0   B0   C0   D0
K1   A1   B1  NaN  NaN
K2   A2   B2   C2   D2
K3  NaN  NaN   C3   D3


In [16]:
print(left.join(right, how="inner"))

     A   B   C   D
K0  A0  B0  C0  D0
K2  A2  B2  C2  D2
